In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import optuna
from tqdm import tqdm

In [2]:
train_data = pd.read_csv('C:\\Users\\akayo\\Hacks\\Alpha_data\\train_1.csv')
test_data = pd.read_csv('C:\\Users\\akayo\\Hacks\\Alpha_data\\test_1.csv')

X = train_data.drop(columns=['id', 'smpl', 'target']).values
y = train_data['target'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [27]:
class ClassifierNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(ClassifierNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [4]:
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs):
    for epoch in tqdm(range(num_epochs)):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                val_preds.extend(outputs.cpu().numpy())
                val_targets.extend(batch_y.cpu().numpy())
        val_loss /= len(val_loader)
        val_auc = roc_auc_score(val_targets, val_preds)

        print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}')

    # Тестирование на тестовых данных
    model.eval()
    test_loss = 0
    test_preds = []
    test_targets = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()
            test_preds.extend(outputs.cpu().numpy())
            test_targets.extend(batch_y.cpu().numpy())
    test_loss /= len(test_loader)
    test_auc = roc_auc_score(test_targets, test_preds)

    print(f'Test Loss: {test_loss:.4f}, Test AUC: {test_auc:.4f}')
    return val_auc, test_auc

In [25]:
def objective(trial):
    input_size = X_train_scaled.shape[1]
    hidden_size1 = trial.suggest_int('hidden_size1', 32, 256)
    hidden_size2 = trial.suggest_int('hidden_size2', 16, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = ClassifierNN(input_size, hidden_size1, hidden_size2)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    val_auc, test_auc = train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=10)
    
    return test_auc

study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=50)

best_params = study.best_params
print(f'Best params: {best_params}')

[I 2024-11-03 14:44:12,715] A new study created in memory with name: no-name-b89d1634-c024-4184-bf51-9c724e20be96
C:\Users\akayo\AppData\Local\Temp\ipykernel_12168\3647203079.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
 10%|█         | 1/10 [00:08<01:16,  8.48s/it]

Epoch [1/10], Val Loss: 0.0563, Val AUC: 0.7936


 20%|██        | 2/10 [00:17<01:08,  8.51s/it]

Epoch [2/10], Val Loss: 0.0557, Val AUC: 0.8038


 30%|███       | 3/10 [00:25<00:58,  8.40s/it]

...

Epoch [9/10], Val Loss: 0.0782, Val AUC: 0.7644


100%|██████████| 10/10 [05:58<00:00, 35.90s/it]

Epoch [10/10], Val Loss: 0.0729, Val AUC: 0.7679



[I 2024-11-03 15:09:12,134] Trial 9 finished with value: 0.7432478016009052 and parameters: {'hidden_size2': 199, 'hidden_size3': 91, 'learning_rate': 0.005298018117263431}. Best is trial 4 with value: 0.7905741325537763.


Test Loss: 0.0753, Test AUC: 0.7432
Best params: {'hidden_size2': 40, 'hidden_size3': 62, 'learning_rate': 1.4403730190525351e-05}


In [12]:
best_model = ClassifierNN(X_train_scaled.shape[1], best_params['hidden_size1'], best_params['hidden_size2'])
criterion = nn.BCELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_model(best_model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=10)

 10%|█         | 1/10 [00:08<01:18,  8.76s/it]

Epoch [1/10], Val Loss: 0.0575, Val AUC: 0.7723


 20%|██        | 2/10 [00:17<01:08,  8.50s/it]

Epoch [2/10], Val Loss: 0.0559, Val AUC: 0.7964


 30%|███       | 3/10 [00:25<00:58,  8.42s/it]

...

Epoch [9/10], Val Loss: 0.0551, Val AUC: 0.8156


100%|██████████| 10/10 [01:24<00:00,  8.41s/it]

Epoch [10/10], Val Loss: 0.0551, Val AUC: 0.8150


Test Loss: 0.0551, Test AUC: 0.8019


(0.8150035072539127, 0.8019381414512876)

In [ ]:
best_model.eval()
with torch.no_grad():
    test_outputs = best_model(X_test_tensor)
    test_predictions = (test_outputs > 0.5).float()

submission = pd.DataFrame({'_ID': test_data['_ID'], 'target': test_predictions.numpy().flatten()})
submission.to_csv('submission.csv', index=False)

print("Предсказания сохранены в файл submission.csv")